In [5]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)    
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def region_of_interest(edges):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    # Next we'll create a masked edges image using cv2.fillPoly()
    #defining a blank mask to start with
    mask = np.zeros_like(edges)   
    ignore_mask_color = 255   

    # This time we are defining a four sided polygon to mask
    imshape = edges.shape
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(imshape) > 2:
        channel_count = imshape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255

    vertices = np.array([[(.51*imshape[1],imshape[0]*.58),(imshape[1]*.49, imshape[0]*.58), 
                          (0, imshape[0] - 50), (imshape[1],imshape[0] - 50)]], dtype=np.int32)
    #filling pixels inside the polygon defined by "vertices" with the fill color 
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    #returning the image only where mask pixels are nonzero
    return cv2.bitwise_and(edges, mask)

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)

def MaskYellowAndWhite(image):
    """
    This function masks 2 colors yellow and white in an image. 
    These 2 colors are very typical for laneline on highways and city roads
    Other colors are filtered out. The yellow or white pixels are preserved
    and converted to gray scale. Pixels in other colors are removed.
    This resulted gray image is the input for the next step - canny edge detection
    """
    # Convert image from BGR to HLS color space
    hls = cv2.cvtColor(image, cv2.COLOR_BGR2HLS).astype(np.float)
    # Mask yellow color in BGR color space of the image
    lower_rgb_yellow = np.array([60,150,150], dtype = "uint8")
    upper_rgb_yellow = np.array([120,250,255], dtype = "uint8")
    mask_rgb_yellow = cv2.inRange(image, lower_rgb_yellow, upper_rgb_yellow)
    rgb_y = cv2.bitwise_and(image, image, mask = mask_rgb_yellow).astype(np.uint8)
    # convert to gray scale
    rgb_y = grayscale(rgb_y)
    # Mask white color in BGR color space of the image
    lower_rgb_white = np.array([190,190,190], dtype = "uint8")
    upper_rgb_white = np.array([255,255,255], dtype = "uint8")
    mask_rgb_white = cv2.inRange(image, lower_rgb_white, upper_rgb_white)
    rgb_w = cv2.bitwise_and(image, image, mask = mask_rgb_white).astype(np.uint8)
    rgb_w = grayscale(rgb_w)
    # Mask yellow color in HLS color space 
    lower_hls_yellow = np.array([20,120,80], dtype = "uint8")
    upper_hls_yellow = np.array([45,200,255], dtype = "uint8")
    mask_hls_yellow = cv2.inRange(hls, lower_hls_yellow, upper_hls_yellow)
    hls_y = cv2.bitwise_and(image, image, mask = mask_hls_yellow).astype(np.uint8)
    hls_y = grayscale(hls_y)
    # sum up the filtered images
    gray = cv2.add(rgb_y, rgb_w)
    gray = cv2.add(gray, hls_y)
    return gray
    
def Draw_Lines(img, lines, slope_Threshold):
    """
    This function draws detected lanelines in an image. All dectected by hough transform
    lines are seperated in 2 groups: left and right lanelines. The slops and centers of all lines
    are calculated. The lines with too small slop will be ignored. For every group average slope 
    and center are determined. This slope and center represent the common detected laneline. 
    Base on the slop and center values 2 end-points of left and right lanelines
    are determined. With this 2 end-points the lanelines will be drawn on the image.
    """
    rm = []
    lm = []
    rc = []
    lc = []
    line_image = np.copy(img)*0 # creating a blank to draw lines on
    imshape = img.shape
    
    if(lines.any()):
        for line in lines:
            # Calculate slope and center of each line
            for x1,y1,x2,y2 in line:
                if (x2 != x1) and (y2 != y1):
                    slop = (y1-y2)/(x1-x2)
                    center = [(x2 + x1)/2, (y2 + y1)/2]
                    if (slop > slope_Threshold):
                        rm.append(slop)
                        rc.append(center)
                    elif (slop < -slope_Threshold):
                        lm.append(slop)
                        lc.append(center)
                        
        # Slope and center of the right laneline
        r_slope = np.sum(rm)/len(rm)
        r_center = np.divide(np.sum(rc, axis = 0), len(rc))
        # determine the right laneline
        if( r_slope != 0) and (len(rm) != 0):
            # determine 2 end-points of the right line
            y1r = imshape[0] * 0.6
            x1r = r_center[0] - ((r_center[1] - y1r)/r_slope)
            y2r = imshape[0]
            x2r = r_center[0] - ((r_center[1] - y2r)/r_slope)
            # draw the line on a blank image
            cv2.line(line_image,(int(x1r),int(y1r)),(int(x2r),int(y2r)),(0,0,255),10)
        else:
            print(" r_slope = 0")
        
        # Slope and center of the left laneline
        l_slope = np.sum(lm)/len(lm)
        l_center = np.divide(np.sum(lc, axis = 0), len(lc)) 
        # determine the left laneline
        if (l_slope != 0) and (len(lm) != 0):
            # determine 2 end-points of the left line
            y1l = imshape[0] * 0.6
            x1l = l_center[0] - ((l_center[1]) - y1l)/(l_slope)
            y2l = imshape[0]
            x2l = l_center[0] - ((l_center[1]) - y2l)/(l_slope)
            # draw the line on a blank image
            cv2.line(line_image,(int(x1l),int(y1l)),(int(x2l),int(y2l)),(0,0,255),10)
        else:
            print(" l_slope = 0")
    else:
        print(" No lines")
        
    return line_image
    
Old_line = []
# Define our parameters for Canny and apply
Canny_low_threshold = 100
Canny_high_threshold = 200
# Define the Hough transform parameters
rho = 1 # distance resolution in pixels of the Hough grid
theta = np.pi/180 # angular resolution in radians of the Hough grid
threshold = 30     # minimum number of votes (intersections in Hough grid cell)
min_line_length = 13 #minimum number of pixels making up a line
max_line_gap = 2    # maximum gap in pixels between connectable line segments

slope_Threshold = 0.17

# cap = cv2.VideoCapture('test_videos/solidYellowLeft.mp4')
# cap = cv2.VideoCapture('test_videos/solidWhiteRight.mp4')
cap = cv2.VideoCapture('test_videos/challenge.mp4')

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:       
        
        # mask yellow and white color
        gray = MaskYellowAndWhite(frame)
        
        gray = gaussian_blur(gray, 3)
        
        edges = canny(gray, Canny_low_threshold, Canny_high_threshold)
        
        masked_edges = region_of_interest(edges)

        # Run Hough on edge detected image
        # Output "lines" is an array containing endpoints of detected line segments
        NewLines = cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]),
                                    min_line_length, max_line_gap)
        
        if len(Old_line) > 0:
            lines = np.concatenate((NewLines,Old_line), axis = 0)
        else:
            lines = np.copy(NewLines)
        
        # Iterate over the output "lines" and draw lines on a blank image
        line_image = Draw_Lines(frame, lines, slope_Threshold)
        
        if len(NewLines) > 0:
            Old_line = np.copy(NewLines)

                
        cv2.imshow('line_image',line_image)

        # Draw the lines on the edge image
        lines_edges = cv2.addWeighted(frame, 0.8, line_image, 1, 0) 

        cv2.imshow('frame',lines_edges)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break               
    else:
        break

        
cap.release()
cv2.destroyAllWindows()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:133: RuntimeWarning: invalid value encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:134: RuntimeWarning: invalid value encountered in true_divide


 r_slope = 0
 r_slope = 0
